In [ ]:
import pymongo
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Connect to MongoDB
client = pymongo.MongoClient("mongodb+srv://admin:<password>@cluster0.jwzyj.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0")
db = client["dining_info"]
collection = db["dining_data"]

# Fetch all records
data = list(collection.find())

# Convert MongoDB data to a DataFrame
df = pd.DataFrame(data)

# Data Cleaning and Transformation
if '_id' in df.columns:
    df.drop('_id', axis=1, inplace=True)

# Convert date columns to datetime format
date_columns = ["check_in_date", "check_out_date", "order_time"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Ensure numerical fields have correct data types
df["age"] = pd.to_numeric(df["age"], errors='coerce')
df["Qty"] = pd.to_numeric(df["Qty"], errors='coerce')
df["price_for_1"] = pd.to_numeric(df["price_for_1"], errors='coerce')
df["number_of_stayers"] = pd.to_numeric(df["number_of_stayers"], errors='coerce')
df["booked_through_points"] = df["booked_through_points"].astype(int)

# Handle missing values
df.fillna({
    "age": df["age"].median(),
    "Qty": df["Qty"].median(),
    "price_for_1": df["price_for_1"].median(),
    "number_of_stayers": df["number_of_stayers"].median(),
}, inplace=True)

# Initialize Dash app
app = dash.Dash(__name__)

def filter_data(start_date, end_date):
    # Convert the start and end date to Timestamps for comparison
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Use .loc to avoid the "SettingWithCopyWarning"
    filtered_df = df.loc[(df['order_time'].dt.date >= start_date.date()) & 
                         (df['order_time'].dt.date <= end_date.date())]
    return filtered_df
    
# Calculate Average Dining Cost by Cuisine
def generate_pie_chart(filtered_df):
    avg_cost_by_cuisine = filtered_df.groupby("Preferred Cusine")["price_for_1"].mean().reset_index()
    return px.pie(avg_cost_by_cuisine, values="price_for_1", names="Preferred Cusine", title="Average Dining Cost by Cuisine")

# Customer Count Over Time
def generate_line_chart(filtered_df):
    filtered_df["order_date"] = filtered_df["order_time"].dt.date
    customer_count = filtered_df.groupby("order_date")["customer_id"].nunique().reset_index()
    return px.line(customer_count, x="order_date", y="customer_id", title="Customer Count Over Time", labels={"customer_id": "Number of Customers", "order_date": "Date"})

# Layout with DatePickerRange for filtering
app.layout = html.Div(children=[
    html.H1("Dining Insights Dashboard", style={"text-align": "center"}),

    html.Div([
        html.H3("Select Date Range"),
        dcc.DatePickerRange(
            id="date-picker-range",
            start_date=df['order_time'].min().date(),
            end_date=df['order_time'].max().date(),
            display_format="YYYY-MM-DD",
            style={"padding": "10px"}
        ),
    ], style={"margin-bottom": "40px"}),

    html.Div([
        html.H3("Average Dining Cost by Cuisine"),
        dcc.Graph(id="pie-chart")
    ], style={"margin-bottom": "40px"}),

    html.Div([
        html.H3("Customer Count Over Time"),
        dcc.Graph(id="line-chart")
    ])
])

# Callback to update the charts based on date range selection
@app.callback(
    [Output("pie-chart", "figure"),
     Output("line-chart", "figure")],
    [Input("date-picker-range", "start_date"),
     Input("date-picker-range", "end_date")]
)
def update_charts(start_date, end_date):
    # Filter data based on the selected date range
    filtered_df = filter_data(start_date, end_date)

    # Generate updated charts
    pie_chart = generate_pie_chart(filtered_df)
    line_chart = generate_line_chart(filtered_df)

    return pie_chart, line_chart

# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True, port=8051)
